# Vacancy Trajectory Analysis

Analyzing vacancy hopping in `VacHopPy` begins with the `Calculator()` factory function and the `Site` class.

---
## The `Calculator()` Factory Function

The `Calculator()` function is the main entry point for `VacHopPy`'s analysis pipeline. It gathers all necessary inputs, performs initial setup, and returns a fully configured `CalculatorEnsemble` object, ready for analysis.

To get started, you call this function as follows:

```bash
calc_ensemble = Calculator(path_traj, site, t_interval)
```
### Key Arguments

* **`path_traj` (*str*)** 

    Path to the trajectory data. This can be either a single HDF5 trajectory file or a directory containing a bundle of HDF5 files. `VacHopPy` will automatically discover and process all valid files in the specified directory.

* **`site` (*Site*)**

    An instance of the `Site` class.

* **`t_interval` (*float, Optional*)**

    The time interval in **picoseconds (ps)** used for averaging atomic positions to determine site occupation. To distinguish true hopping events from thermal vibrations, `VacHopPy` averages the atomic coordinates and forces over each `t_interval`. This averaged quantities are then used to assign an atom to a specific lattice site. Each interval corresponds to a single analysis step. If this argument is not provided, `Calculator()` will automatically search for an optimal `t_interval` for the analysis.

### Return Value

Calling the `Calculator()` function returns an instance of the `CalculatorEnsemble` class, which holds all the processed data and methods required for subsequent analysis steps.

---
## The `Site` Class

The `Site` class is used to define the structural backbone of your material. It identifies all **lattice sites** from a vacancy-free reference structure and determines the possible **hopping paths** between them.

You can create a `Site` object like this:

```bash
site = Site(path_structure, symbol)
```

### Key Arguments

* **`path_structure` (*str*)**

    Path to a structure file of the perfect, vacancy-free material. Any format supported by the **Atomic Simulation Environment (ASE)** is compatible.

* **`symbol` (*str*)**

    The chemical symbol of the diffusing species.

----

## Usage Example

You can download source files to follow this tutorial from this [link](https://drive.google.com/file/d/1xBj3iP4eUInB2OKxCHstel4CTyXUDU9t/view?usp=sharing)


Navigate into the `Example6/` directory you downloaded. In this directory, you will find two files:

* **`TRAJ_O.h5`**

    This is the HDF5 file containing the MD trajectory data. The example system is **rutile TiO₂** containing **two oxygen vacancies**, simulated at **2100 K**.

* **`POSCAR_TiO2`**

    This file contains the crystal structure of the perfect, **vacancy-free** rutile rutile TiO₂ supercell.


In [1]:
import os
import numpy as np
from vachoppy.utils import show_traj
from vachoppy.core import Site, Calculator

path_traj, path_structure = 'TRAJ_O.h5', 'POSCAR_TiO2'

if not os.path.exists(path_traj): print(f"{path_traj} not found.")
if not os.path.exists(path_structure): print(f"{path_structure} not found.")

---
### 1. How to inspect the HDF5 file

You can inspect the contents of the HDF5 trajectory file using the `show_traj` method:

In [2]:
show_traj(path_traj)

  Trajectory File: TRAJ_O.h5

[Simulation Parameters]
  - Atomic Symbol:      O
  - Number of Frames:   300000
  - Temperature:        2100.0 K
  - Time Step:          2.0 fs

[Composition]
  - Counts:             O: 46, Ti: 24
  - Total Atoms:        70

[Lattice Vectors (Ang)]
  [  9.29133,   0.00000,   0.00000]
  [  0.00000,   9.29133,   0.00000]
  [  0.00000,   0.00000,   8.90126]

[Stored Datasets]
  - positions:          Shape = (300000, 46, 3)
  - forces:             Shape = (300000, 46, 3)


From the output, the MD run was performed at 2100 K for 600 ps (300,000 iterations × 2.0 fs). The composition shows the system contains **two oxygen vacancies**.

---
### 2. Creating a `Site` Instance

In [3]:
site = Site(path_structure, 'O')

You can inspect the generated lattice information by calling the `.summary()` method.

`VacHopPy` automatically identifies all possible hopping paths by employing the **Voronoi tessellation** technique. The maximum distance for this search can be adjusted using the `rmax` parameter, which defaults to **3.25 Å**.

In [5]:
site.summary()


  Structure File: POSCAR_TiO2
[Structure Information]
    - Structure Composition : Ti24 O48
    - Lattice Vectors (Ang) :
        [  9.29133,   0.00000,   0.00000]
        [  0.00000,   9.29133,   0.00000]
        [  0.00000,   0.00000,   8.90126]

[Hopping Path Information]
    - Diffusing Symbol   : O
    - Inequivalent Sites : 1 found
    - Inequivalent Paths : 3 found (with Rmax = 3.25 Å)

Name    Init Site    Final Site    a (Å)    z    Initial Coord (Frac)      Final Coord (Frac)
------  -----------  ------------  -------  ---  ------------------------  -------------------------
A1      site1        site1         2.5625   1    [0.0975, 0.4025, 0.1667]  [-0.0975, 0.5975, 0.1667]
A2      site1        site1         2.8031   8    [0.0975, 0.4025, 0.1667]  [0.3475, 0.3475, 0.0000]
A3      site1        site1         2.9671   2    [0.0975, 0.4025, 0.1667]  [0.0975, 0.4025, -0.1667]



---
### 3. Creating the `CalculatorEnsemble` Instance

While you can create a `CalculatorEnsemble` instance by calling its constructor directly, the recommended approach is to use the `Calculator()` factory function. This helper function offers a significant advantage: it can automatically search for an optimal `t_interval` if one is not provided, streamlining your analysis setup

In [6]:
calc_ensemble = Calculator(path_traj, site)

                   Automatic t_interval Estimation
  Estimating from TRAJ_O.h5
             -> t_interval : 0.076 ps
        Adjusting t_interval to the nearest multiple of dt
    - dt                  : 0.0020 ps
    - Original t_interval : 0.0764 ps
    - Adjusted t_interval : 0.0760 ps (38 frames)


Now that the `calc_ensemble` object is prepared, you can kick off the main analysis by calling the `.calculate()` method.

This powerful method handles the entire analysis pipeline for you, which automatically includes **identifying the trajectory of each vacancy** and **calculating hopping parameters**.

In [7]:
calc_ensemble.calculate()

Analyze Trajectory:   0%|                              | 0/1 [00:00<?, ?it/s]


Analysis complete: 1 successful, 0 failed.
Execution Time: 2.528 seconds
Peak RAM Usage: 0.043 GB


Although this tutorial uses a single HDF5 file, the `CalculatorEnsemble` class is specifically designed to process multiple trajectory files at once.

The results for each individual HDF5 file are stored in the `.calculators` attribute. This attribute is a list of `CalculatorSingle` objects, sorted first by **temperature** (ascending) and then **alphabetically by file path**.

You can verify which `CalculatorSingle` instance corresponds to which HDF5 file using the following code:

In [8]:
for i, calc in enumerate(calc_ensemble.calculators):
    print(f"calculators[{i}] : {calc.path_traj}")

calculators[0] : /home/jty/Examples/Example6/TRAJ_O.h5


For the remainder of this tutorial, we will focus on the results from the first trajectory by selecting its corresponding object.

In [ ]:
calc = calc_ensemble.calculators[0]

Key analysis parameters are now stored as attributes in the `calc` object and can be viewed as follows:

In [9]:
print(f"- Number of Vacancies                       : {calc.num_vacancies}")
print(f"- Time Step for the MD Simulation (dt)      : {calc.dt} fs")
print(f"- Time interval for Averaging  (t_interval) : {calc.t_interval} ps")
print(f"- Number of Frames in the MD Simulatoin     : {calc.num_frames} frames")
print(f"- Number of Frames per Step                 : {int(calc.t_interval * 1000 / calc.dt)} frames")
print(f"- Number of Steps for Analysis              : {calc.num_steps} steps")

- Number of Vacancies                       : 2
- Time Step for the MD Simulation (dt)      : 2.0 fs
- Time interval for Averaging  (t_interval) : 0.076 ps
- Number of Frames in the MD Simulatoin     : 299972 frames
- Number of Frames per Step                 : 38 frames
- Number of Steps for Analysis              : 7894 steps


You might notice that while the MD simulation contains 300,000 frames, only 299,972 are used in the analysis. This is a direct consequence of the time-averaging process controlled by `t_interval`.

`VacHopPy` groups the raw simulation frames into blocks, with each block corresponding to a single analysis step. The duration of each step is defined by `t_interval`.

In this example, with a `t_interval` of 0.076 ps (or 76 fs) and a simulation timestep (`dt`) of 2 fs, each analysis step is derived by averaging **76 fs / 2 fs = 38** frames.

Because the analysis requires complete steps, `VacHopPy` uses the largest number of frames that is perfectly divisible by 38. Any remaining frames at the end of the trajectory are discarded. Therefore, the total number of analysis steps is **int(300000 / 38) = 7894**, and the total frames used is **7894 * 38 = 299972**.

---
### Analyzing Vacancy Hopping Trajectories

Once the calculation is complete, you can access the detailed hopping trajectory of each vacancy through two key attributes. These attributes provide different but complementary views of the vacancy movement.

* **`.hopping_history` (*list*)**

    A list where the length is equal to the number of vacancies (`num_vacancies`). Each element in the list corresponds to a single vacancy and contains its complete hopping history.

* **`.unwrapped_vacancy_trajectory_coord_cart` (*dict*)**

    A dictionary where each key is an analysis step (an integer) and the value contains the corresponding spatial coordinates. The coordinates are the **PBC-unwrapped Cartesian positions (in Å)** of all vacancies at that specific step. 


#### Usage of `.hopping_history`

Let's examine the history of the first vacancy (`calc.hopping_history[0]`) to see this in action:

In [23]:
print(f"Number of Vacancies: {len(calc.hopping_history)}")
print(f"Number of Hopping Events for the First Vacancy: {len(calc.hopping_history[0])}")

Number of Vacancies: 2
Number of Hopping Events for the First Vacancy: 111


The raw dictionary output for a single event can be hard to read. We can easily format it for better clarity:

In [26]:
# Get the last hopping event of the first vacancy
last_event = calc.hopping_history[0][-1]

print("\n--- Details of the Last Hopping Event ---")
for key, value in last_event.items():
    if 'coord' in key and isinstance(value, np.ndarray):
         print(f"- {key:<12}: [{value[0]:.3f}, {value[1]:.3f}, {value[2]:.3f}]")
    else:
         print(f"- {key:<12}: {value}")


--- Details of the Last Hopping Event ---
- site_init   : site1
- site_final  : site1
- distance    : 2.5624908680896756
- z           : 1
- coord_init  : [0.098, 0.402, 0.167]
- coord_final : [-0.098, 0.598, 0.167]
- name        : A1
- step        : 7825
- index_init  : 24
- index_final : 36


#### Usage of `.unwrapped_vacancy_trajectory_coord_cart`

To get the coordinates at the 100th step:

In [29]:
coords_at_step_100 = calc.unwrapped_vacancy_trajectory_coord_cart[100]
print(coords_at_step_100)

[[0.90597733 3.73968856 1.48354252]
 [3.22881028 3.22881028 2.96708503]]


The result is a NumPy array where each row represents the **[x, y, z]** coordinates of a single vacancy at the specified step. The first row corresponds to the first vacancy, the second row to the second, and so on. 

#### Save the Vacancy Hopping Histories

You can save this unwrapped trajectory into the `trajectory.json` file using `.save_trajectory()` method.

In [43]:
calc.save_trajectory()

#### Summary of Vacancy Hopping Histories

You can print the hopping history of each vacancy easily using `.show_hopping_history()`:

In [ ]:
calc.show_hopping_history()

```{code-block} bash
:class: scrollable-output

====================================================================================================================
                                           Hopping Sequence of Vacancy 0
====================================================================================================================
Num    Time (ps)    Path    a (Ang)    Initial Site (Fractional Coordinate)    Final Site (Fractional Coordinate)
-----  -----------  ------  ---------  --------------------------------------  ------------------------------------
1      11.93        A2      2.80311    site1 [0.09751, 0.40249, 0.16667]       site1 [0.15249, 0.15249, 0.33333]
2      13.6         A1      2.56249    site1 [0.15249, 0.15249, 0.33333]       site1 [0.34751, 0.34751, 0.33333]
3      14.44        A1      2.56249    site1 [0.34751, 0.34751, 0.33333]       site1 [0.15249, 0.15249, 0.33333]
4      15.43        A1      2.56249    site1 [0.15249, 0.15249, 0.33333]       site1 [0.34751, 0.34751, 0.33333]
5      17.02        A1      2.56249    site1 [0.34751, 0.34751, 0.33333]       site1 [0.15249, 0.15249, 0.33333]
6      20.44        A1      2.56249    site1 [0.15249, 0.15249, 0.33333]       site1 [0.34751, 0.34751, 0.33333]
7      22.34        A1      2.56249    site1 [0.34751, 0.34751, 0.33333]       site1 [0.15249, 0.15249, 0.33333]
8      22.95        A1      2.56249    site1 [0.15249, 0.15249, 0.33333]       site1 [0.34751, 0.34751, 0.33333]
9      23.26        A1      2.56249    site1 [0.34751, 0.34751, 0.33333]       site1 [0.15249, 0.15249, 0.33333]
10     24.24        A1      2.56249    site1 [0.15249, 0.15249, 0.33333]       site1 [0.34751, 0.34751, 0.33333]
11     25.38        A1      2.56249    site1 [0.34751, 0.34751, 0.33333]       site1 [0.15249, 0.15249, 0.33333]
12     26.9         A1      2.56249    site1 [0.15249, 0.15249, 0.33333]       site1 [0.34751, 0.34751, 0.33333]
13     28.58        A1      2.56249    site1 [0.34751, 0.34751, 0.33333]       site1 [0.15249, 0.15249, 0.33333]
14     29.26        A1      2.56249    site1 [0.15249, 0.15249, 0.33333]       site1 [0.34751, 0.34751, 0.33333]
15     38.3         A1      2.56249    site1 [0.34751, 0.34751, 0.33333]       site1 [0.15249, 0.15249, 0.33333]
16     38.84        A1      2.56249    site1 [0.15249, 0.15249, 0.33333]       site1 [0.34751, 0.34751, 0.33333]
17     39.82        A1      2.56249    site1 [0.34751, 0.34751, 0.33333]       site1 [0.15249, 0.15249, 0.33333]
18     40.28        A1      2.56249    site1 [0.15249, 0.15249, 0.33333]       site1 [0.34751, 0.34751, 0.33333]
19     41.19        A1      2.56249    site1 [0.34751, 0.34751, 0.33333]       site1 [0.15249, 0.15249, 0.33333]
20     41.57        A1      2.56249    site1 [0.15249, 0.15249, 0.33333]       site1 [0.34751, 0.34751, 0.33333]
21     42.18        A1      2.56249    site1 [0.34751, 0.34751, 0.33333]       site1 [0.15249, 0.15249, 0.33333]
22     44.92        A2      2.80311    site1 [0.15249, 0.15249, 0.33333]       site1 [0.09751, 0.90249, 0.16667]
23     54.34        A2      2.80311    site1 [0.09751, 0.90249, 0.16667]       site1 [0.84751, 0.84751, 0.00000]
24     98.65        A2      2.80311    site1 [0.84751, 0.84751, 0.00000]       site1 [0.90249, 0.59751, 0.16667]
25     108.22       A1      2.56249    site1 [0.90249, 0.59751, 0.16667]       site1 [0.09751, 0.40249, 0.16667]
26     163.1        A1      2.56249    site1 [0.09751, 0.40249, 0.16667]       site1 [0.90249, 0.59751, 0.16667]
27     166.21       A1      2.56249    site1 [0.90249, 0.59751, 0.16667]       site1 [0.09751, 0.40249, 0.16667]
28     174.5        A2      2.80311    site1 [0.09751, 0.40249, 0.16667]       site1 [0.15249, 0.65249, 0.00000]
29     195.55       A1      2.56249    site1 [0.15249, 0.65249, 0.00000]       site1 [0.34751, 0.84751, 0.00000]
30     196.16       A1      2.56249    site1 [0.34751, 0.84751, 0.00000]       site1 [0.15249, 0.65249, 0.00000]
31     196.99       A1      2.56249    site1 [0.15249, 0.65249, 0.00000]       site1 [0.34751, 0.84751, 0.00000]
32     197.14       A1      2.56249    site1 [0.34751, 0.84751, 0.00000]       site1 [0.15249, 0.65249, 0.00000]
33     197.6        A1      2.56249    site1 [0.15249, 0.65249, 0.00000]       site1 [0.34751, 0.84751, 0.00000]
34     197.98       A1      2.56249    site1 [0.34751, 0.84751, 0.00000]       site1 [0.15249, 0.65249, 0.00000]
35     198.74       A1      2.56249    site1 [0.15249, 0.65249, 0.00000]       site1 [0.34751, 0.84751, 0.00000]
36     198.97       A1      2.56249    site1 [0.34751, 0.84751, 0.00000]       site1 [0.15249, 0.65249, 0.00000]
37     199.5        A1      2.56249    site1 [0.15249, 0.65249, 0.00000]       site1 [0.34751, 0.84751, 0.00000]
38     202.84       A1      2.56249    site1 [0.34751, 0.84751, 0.00000]       site1 [0.15249, 0.65249, 0.00000]
39     204.74       A1      2.56249    site1 [0.15249, 0.65249, 0.00000]       site1 [0.34751, 0.84751, 0.00000]
40     205.05       A2      2.80311    site1 [0.34751, 0.84751, 0.00000]       site1 [0.09751, 0.90249, 0.16667]
41     212.72       A2      2.80311    site1 [0.09751, 0.90249, 0.16667]       site1 [0.84751, 0.84751, 0.33333]
42     213.18       A1      2.56249    site1 [0.84751, 0.84751, 0.33333]       site1 [0.65249, 0.65249, 0.33333]
43     215.16       A1      2.56249    site1 [0.65249, 0.65249, 0.33333]       site1 [0.84751, 0.84751, 0.33333]
44     219.41       A2      2.80311    site1 [0.84751, 0.84751, 0.33333]       site1 [0.90249, 0.09751, 0.50000]
45     232.71       A1      2.56249    site1 [0.90249, 0.09751, 0.50000]       site1 [0.09751, 0.90249, 0.50000]
46     243.43       A2      2.80311    site1 [0.09751, 0.90249, 0.50000]       site1 [0.34751, 0.84751, 0.66667]
47     245.18       A1      2.56249    site1 [0.34751, 0.84751, 0.66667]       site1 [0.15249, 0.65249, 0.66667]
48     249.05       A2      2.80311    site1 [0.15249, 0.65249, 0.66667]       site1 [0.40249, 0.59751, 0.50000]
49     250.27       A2      2.80311    site1 [0.40249, 0.59751, 0.50000]       site1 [0.15249, 0.65249, 0.66667]
50     252.4        A1      2.56249    site1 [0.15249, 0.65249, 0.66667]       site1 [0.34751, 0.84751, 0.66667]
51     253          A1      2.56249    site1 [0.34751, 0.84751, 0.66667]       site1 [0.15249, 0.65249, 0.66667]
52     253.46       A1      2.56249    site1 [0.15249, 0.65249, 0.66667]       site1 [0.34751, 0.84751, 0.66667]
53     253.84       A1      2.56249    site1 [0.34751, 0.84751, 0.66667]       site1 [0.15249, 0.65249, 0.66667]
54     253.92       A1      2.56249    site1 [0.15249, 0.65249, 0.66667]       site1 [0.34751, 0.84751, 0.66667]
55     256.35       A1      2.56249    site1 [0.34751, 0.84751, 0.66667]       site1 [0.15249, 0.65249, 0.66667]
56     257.18       A1      2.56249    site1 [0.15249, 0.65249, 0.66667]       site1 [0.34751, 0.84751, 0.66667]
57     257.64       A2      2.80311    site1 [0.34751, 0.84751, 0.66667]       site1 [0.40249, 0.09751, 0.83333]
58     257.72       A2      2.80311    site1 [0.40249, 0.09751, 0.83333]       site1 [0.34751, 0.84751, 0.66667]
59     262.58       A2      2.80311    site1 [0.34751, 0.84751, 0.66667]       site1 [0.40249, 0.59751, 0.83333]
60     270.41       A2      2.80311    site1 [0.40249, 0.59751, 0.83333]       site1 [0.34751, 0.34751, 0.66667]
61     270.94       A2      2.80311    site1 [0.34751, 0.34751, 0.66667]       site1 [0.09751, 0.40249, 0.83333]
62     271.4        A1      2.56249    site1 [0.09751, 0.40249, 0.83333]       site1 [0.90249, 0.59751, 0.83333]
63     273.3        A1      2.56249    site1 [0.90249, 0.59751, 0.83333]       site1 [0.09751, 0.40249, 0.83333]
64     273.98       A1      2.56249    site1 [0.09751, 0.40249, 0.83333]       site1 [0.90249, 0.59751, 0.83333]
65     290.85       A1      2.56249    site1 [0.90249, 0.59751, 0.83333]       site1 [0.09751, 0.40249, 0.83333]
66     291.46       A1      2.56249    site1 [0.09751, 0.40249, 0.83333]       site1 [0.90249, 0.59751, 0.83333]
67     313.42       A2      2.80311    site1 [0.90249, 0.59751, 0.83333]       site1 [0.84751, 0.84751, 0.66667]
68     316.16       A1      2.56249    site1 [0.84751, 0.84751, 0.66667]       site1 [0.65249, 0.65249, 0.66667]
69     316.77       A1      2.56249    site1 [0.65249, 0.65249, 0.66667]       site1 [0.84751, 0.84751, 0.66667]
70     334.86       A1      2.56249    site1 [0.84751, 0.84751, 0.66667]       site1 [0.65249, 0.65249, 0.66667]
71     346.1        A2      2.80311    site1 [0.65249, 0.65249, 0.66667]       site1 [0.90249, 0.59751, 0.50000]
72     352.03       A2      2.80311    site1 [0.90249, 0.59751, 0.50000]       site1 [0.84751, 0.34751, 0.33333]
73     355.53       A2      2.80311    site1 [0.84751, 0.34751, 0.33333]       site1 [0.09751, 0.40249, 0.50000]
74     357.12       A1      2.56249    site1 [0.09751, 0.40249, 0.50000]       site1 [0.90249, 0.59751, 0.50000]
75     357.81       A1      2.56249    site1 [0.90249, 0.59751, 0.50000]       site1 [0.09751, 0.40249, 0.50000]
76     358.04       A1      2.56249    site1 [0.09751, 0.40249, 0.50000]       site1 [0.90249, 0.59751, 0.50000]
77     359.48       A1      2.56249    site1 [0.90249, 0.59751, 0.50000]       site1 [0.09751, 0.40249, 0.50000]
78     359.94       A1      2.56249    site1 [0.09751, 0.40249, 0.50000]       site1 [0.90249, 0.59751, 0.50000]
79     360.01       A1      2.56249    site1 [0.90249, 0.59751, 0.50000]       site1 [0.09751, 0.40249, 0.50000]
80     360.32       A1      2.56249    site1 [0.09751, 0.40249, 0.50000]       site1 [0.90249, 0.59751, 0.50000]
81     361.23       A1      2.56249    site1 [0.90249, 0.59751, 0.50000]       site1 [0.09751, 0.40249, 0.50000]
82     362.6        A1      2.56249    site1 [0.09751, 0.40249, 0.50000]       site1 [0.90249, 0.59751, 0.50000]
83     366.47       A1      2.56249    site1 [0.90249, 0.59751, 0.50000]       site1 [0.09751, 0.40249, 0.50000]
84     368.52       A1      2.56249    site1 [0.09751, 0.40249, 0.50000]       site1 [0.90249, 0.59751, 0.50000]
85     373.16       A1      2.56249    site1 [0.90249, 0.59751, 0.50000]       site1 [0.09751, 0.40249, 0.50000]
86     373.39       A1      2.56249    site1 [0.09751, 0.40249, 0.50000]       site1 [0.90249, 0.59751, 0.50000]
87     377.8        A2      2.80311    site1 [0.90249, 0.59751, 0.50000]       site1 [0.84751, 0.34751, 0.66667]
88     380.61       A1      2.56249    site1 [0.84751, 0.34751, 0.66667]       site1 [0.65249, 0.15249, 0.66667]
89     386.76       A1      2.56249    site1 [0.65249, 0.15249, 0.66667]       site1 [0.84751, 0.34751, 0.66667]
90     389.58       A2      2.80311    site1 [0.84751, 0.34751, 0.66667]       site1 [0.90249, 0.59751, 0.83333]
91     389.73       A2      2.80311    site1 [0.90249, 0.59751, 0.83333]       site1 [0.84751, 0.84751, 0.00000]
92     390.26       A1      2.56249    site1 [0.84751, 0.84751, 0.00000]       site1 [0.65249, 0.65249, 0.00000]
93     392.08       A1      2.56249    site1 [0.65249, 0.65249, 0.00000]       site1 [0.84751, 0.84751, 0.00000]
94     392.39       A2      2.80311    site1 [0.84751, 0.84751, 0.00000]       site1 [0.59751, 0.90249, 0.16667]
95     408.27       A2      2.80311    site1 [0.59751, 0.90249, 0.16667]       site1 [0.65249, 0.65249, 0.00000]
96     408.42       A2      2.80311    site1 [0.65249, 0.65249, 0.00000]       site1 [0.90249, 0.59751, 0.83333]
97     417.85       A2      2.80311    site1 [0.90249, 0.59751, 0.83333]       site1 [0.84751, 0.84751, 0.00000]
98     418.68       A2      2.80311    site1 [0.84751, 0.84751, 0.00000]       site1 [0.90249, 0.59751, 0.83333]
99     451.29       A1      2.56249    site1 [0.90249, 0.59751, 0.83333]       site1 [0.09751, 0.40249, 0.83333]
100    462.99       A2      2.80311    site1 [0.09751, 0.40249, 0.83333]       site1 [0.15249, 0.15249, 0.66667]
101    475.08       A2      2.80311    site1 [0.15249, 0.15249, 0.66667]       site1 [0.09751, 0.40249, 0.83333]
102    486.78       A1      2.56249    site1 [0.09751, 0.40249, 0.83333]       site1 [0.90249, 0.59751, 0.83333]
103    508.52       A1      2.56249    site1 [0.90249, 0.59751, 0.83333]       site1 [0.09751, 0.40249, 0.83333]
104    515.2        A1      2.56249    site1 [0.09751, 0.40249, 0.83333]       site1 [0.90249, 0.59751, 0.83333]
105    517.33       A2      2.80311    site1 [0.90249, 0.59751, 0.83333]       site1 [0.84751, 0.84751, 0.00000]
106    522.58       A2      2.80311    site1 [0.84751, 0.84751, 0.00000]       site1 [0.59751, 0.90249, 0.16667]
107    546.06       A1      2.56249    site1 [0.59751, 0.90249, 0.16667]       site1 [0.40249, 0.09751, 0.16667]
108    552.9        A1      2.56249    site1 [0.40249, 0.09751, 0.16667]       site1 [0.59751, 0.90249, 0.16667]
109    573.57       A1      2.56249    site1 [0.59751, 0.90249, 0.16667]       site1 [0.40249, 0.09751, 0.16667]
110    583.15       A2      2.80311    site1 [0.40249, 0.09751, 0.16667]       site1 [0.15249, 0.15249, 0.00000]
111    594.7        A1      2.56249    site1 [0.15249, 0.15249, 0.00000]       site1 [0.34751, 0.34751, 0.00000]
====================================================================================================================

====================================================================================================================
                                           Hopping Sequence of Vacancy 1
====================================================================================================================
Num    Time (ps)    Path    a (Ang)    Initial Site (Fractional Coordinate)    Final Site (Fractional Coordinate)
-----  -----------  ------  ---------  --------------------------------------  ------------------------------------
1      7.3          A2      2.80311    site1 [0.40249, 0.59751, 0.50000]       site1 [0.34751, 0.34751, 0.33333]
2      8.13         A1      2.56249    site1 [0.34751, 0.34751, 0.33333]       site1 [0.15249, 0.15249, 0.33333]
3      8.36         A1      2.56249    site1 [0.15249, 0.15249, 0.33333]       site1 [0.34751, 0.34751, 0.33333]
4      8.59         A1      2.56249    site1 [0.34751, 0.34751, 0.33333]       site1 [0.15249, 0.15249, 0.33333]
5      9.2          A1      2.56249    site1 [0.15249, 0.15249, 0.33333]       site1 [0.34751, 0.34751, 0.33333]
6      9.42         A1      2.56249    site1 [0.34751, 0.34751, 0.33333]       site1 [0.15249, 0.15249, 0.33333]
7      10.18        A1      2.56249    site1 [0.15249, 0.15249, 0.33333]       site1 [0.34751, 0.34751, 0.33333]
8      11.4         A1      2.56249    site1 [0.34751, 0.34751, 0.33333]       site1 [0.15249, 0.15249, 0.33333]
9      11.86        A1      2.56249    site1 [0.15249, 0.15249, 0.33333]       site1 [0.34751, 0.34751, 0.33333]
10     12.16        A2      2.80311    site1 [0.34751, 0.34751, 0.33333]       site1 [0.40249, 0.09751, 0.50000]
11     24.7         A2      2.80311    site1 [0.40249, 0.09751, 0.50000]       site1 [0.15249, 0.15249, 0.33333]
12     24.93        A2      2.80311    site1 [0.15249, 0.15249, 0.33333]       site1 [0.40249, 0.09751, 0.50000]
13     29.26        A1      2.56249    site1 [0.40249, 0.09751, 0.50000]       site1 [0.59751, 0.90249, 0.50000]
14     37.77        A1      2.56249    site1 [0.59751, 0.90249, 0.50000]       site1 [0.40249, 0.09751, 0.50000]
15     43.09        A1      2.56249    site1 [0.40249, 0.09751, 0.50000]       site1 [0.59751, 0.90249, 0.50000]
16     54.11        A2      2.80311    site1 [0.59751, 0.90249, 0.50000]       site1 [0.65249, 0.65249, 0.33333]
17     77.44        A1      2.56249    site1 [0.65249, 0.65249, 0.33333]       site1 [0.84751, 0.84751, 0.33333]
18     96.52        A2      2.80311    site1 [0.84751, 0.84751, 0.33333]       site1 [0.90249, 0.59751, 0.16667]
19     98.57        A2      2.80311    site1 [0.90249, 0.59751, 0.16667]       site1 [0.65249, 0.65249, 0.00000]
20     98.95        A1      2.56249    site1 [0.65249, 0.65249, 0.00000]       site1 [0.84751, 0.84751, 0.00000]
21     100.02       A1      2.56249    site1 [0.84751, 0.84751, 0.00000]       site1 [0.65249, 0.65249, 0.00000]
22     100.7        A1      2.56249    site1 [0.65249, 0.65249, 0.00000]       site1 [0.84751, 0.84751, 0.00000]
23     103.36       A1      2.56249    site1 [0.84751, 0.84751, 0.00000]       site1 [0.65249, 0.65249, 0.00000]
24     103.44       A1      2.56249    site1 [0.65249, 0.65249, 0.00000]       site1 [0.84751, 0.84751, 0.00000]
25     103.66       A1      2.56249    site1 [0.84751, 0.84751, 0.00000]       site1 [0.65249, 0.65249, 0.00000]
26     104.27       A1      2.56249    site1 [0.65249, 0.65249, 0.00000]       site1 [0.84751, 0.84751, 0.00000]
27     106.48       A1      2.56249    site1 [0.84751, 0.84751, 0.00000]       site1 [0.65249, 0.65249, 0.00000]
28     113.32       A1      2.56249    site1 [0.65249, 0.65249, 0.00000]       site1 [0.84751, 0.84751, 0.00000]
29     118.64       A2      2.80311    site1 [0.84751, 0.84751, 0.00000]       site1 [0.90249, 0.59751, 0.83333]
30     162.79       A2      2.80311    site1 [0.90249, 0.59751, 0.83333]       site1 [0.65249, 0.65249, 0.00000]
31     164.39       A1      2.56249    site1 [0.65249, 0.65249, 0.00000]       site1 [0.84751, 0.84751, 0.00000]
32     164.62       A1      2.56249    site1 [0.84751, 0.84751, 0.00000]       site1 [0.65249, 0.65249, 0.00000]
33     165.45       A1      2.56249    site1 [0.65249, 0.65249, 0.00000]       site1 [0.84751, 0.84751, 0.00000]
34     192.36       A2      2.80311    site1 [0.84751, 0.84751, 0.00000]       site1 [0.90249, 0.09751, 0.83333]
35     194.48       A1      2.56249    site1 [0.90249, 0.09751, 0.83333]       site1 [0.09751, 0.90249, 0.83333]
36     206.95       A2      2.80311    site1 [0.09751, 0.90249, 0.83333]       site1 [0.15249, 0.65249, 0.66667]
37     208.09       A1      2.56249    site1 [0.15249, 0.65249, 0.66667]       site1 [0.34751, 0.84751, 0.66667]
38     208.16       A2      2.80311    site1 [0.34751, 0.84751, 0.66667]       site1 [0.59751, 0.90249, 0.50000]
39     215.54       A2      2.80311    site1 [0.59751, 0.90249, 0.50000]       site1 [0.65249, 0.65249, 0.33333]
40     215.84       A2      2.80311    site1 [0.65249, 0.65249, 0.33333]       site1 [0.59751, 0.90249, 0.50000]
41     216.22       A2      2.80311    site1 [0.59751, 0.90249, 0.50000]       site1 [0.34751, 0.84751, 0.66667]
42     231.65       A1      2.56249    site1 [0.34751, 0.84751, 0.66667]       site1 [0.15249, 0.65249, 0.66667]
43     232.71       A1      2.56249    site1 [0.15249, 0.65249, 0.66667]       site1 [0.34751, 0.84751, 0.66667]
44     233.17       A1      2.56249    site1 [0.34751, 0.84751, 0.66667]       site1 [0.15249, 0.65249, 0.66667]
45     233.62       A1      2.56249    site1 [0.15249, 0.65249, 0.66667]       site1 [0.34751, 0.84751, 0.66667]
46     233.7        A1      2.56249    site1 [0.34751, 0.84751, 0.66667]       site1 [0.15249, 0.65249, 0.66667]
47     234.08       A1      2.56249    site1 [0.15249, 0.65249, 0.66667]       site1 [0.34751, 0.84751, 0.66667]
48     234.16       A1      2.56249    site1 [0.34751, 0.84751, 0.66667]       site1 [0.15249, 0.65249, 0.66667]
49     234.61       A1      2.56249    site1 [0.15249, 0.65249, 0.66667]       site1 [0.34751, 0.84751, 0.66667]
50     238.11       A1      2.56249    site1 [0.34751, 0.84751, 0.66667]       site1 [0.15249, 0.65249, 0.66667]
51     238.64       A1      2.56249    site1 [0.15249, 0.65249, 0.66667]       site1 [0.34751, 0.84751, 0.66667]
52     238.79       A1      2.56249    site1 [0.34751, 0.84751, 0.66667]       site1 [0.15249, 0.65249, 0.66667]
53     241.38       A1      2.56249    site1 [0.15249, 0.65249, 0.66667]       site1 [0.34751, 0.84751, 0.66667]
54     241.68       A2      2.80311    site1 [0.34751, 0.84751, 0.66667]       site1 [0.40249, 0.59751, 0.83333]
55     246.7        A2      2.80311    site1 [0.40249, 0.59751, 0.83333]       site1 [0.34751, 0.84751, 0.66667]
56     247.23       A2      2.80311    site1 [0.34751, 0.84751, 0.66667]       site1 [0.09751, 0.90249, 0.83333]
57     248.75       A2      2.80311    site1 [0.09751, 0.90249, 0.83333]       site1 [0.34751, 0.84751, 0.66667]
58     250.34       A2      2.80311    site1 [0.34751, 0.84751, 0.66667]       site1 [0.40249, 0.59751, 0.50000]
59     257.79       A2      2.80311    site1 [0.40249, 0.59751, 0.50000]       site1 [0.15249, 0.65249, 0.66667]
60     258.02       A2      2.80311    site1 [0.15249, 0.65249, 0.66667]       site1 [0.40249, 0.59751, 0.50000]
61     260.38       A1      2.56249    site1 [0.40249, 0.59751, 0.50000]       site1 [0.59751, 0.40249, 0.50000]
62     269.12       A2      2.80311    site1 [0.59751, 0.40249, 0.50000]       site1 [0.84751, 0.34751, 0.33333]
63     277.25       A1      2.56249    site1 [0.84751, 0.34751, 0.33333]       site1 [0.65249, 0.15249, 0.33333]
64     287.2        A1      2.56249    site1 [0.65249, 0.15249, 0.33333]       site1 [0.84751, 0.34751, 0.33333]
65     293.89       A1      2.56249    site1 [0.84751, 0.34751, 0.33333]       site1 [0.65249, 0.15249, 0.33333]
66     313.27       A2      2.80311    site1 [0.65249, 0.15249, 0.33333]       site1 [0.40249, 0.09751, 0.50000]
67     317          A2      2.80311    site1 [0.40249, 0.09751, 0.50000]       site1 [0.65249, 0.15249, 0.33333]
68     334.63       A1      2.56249    site1 [0.65249, 0.15249, 0.33333]       site1 [0.84751, 0.34751, 0.33333]
69     351.96       A2      2.80311    site1 [0.84751, 0.34751, 0.33333]       site1 [0.09751, 0.40249, 0.50000]
70     352.64       A1      2.56249    site1 [0.09751, 0.40249, 0.50000]       site1 [0.90249, 0.59751, 0.50000]
71     352.94       A1      2.56249    site1 [0.90249, 0.59751, 0.50000]       site1 [0.09751, 0.40249, 0.50000]
72     353.55       A1      2.56249    site1 [0.09751, 0.40249, 0.50000]       site1 [0.90249, 0.59751, 0.50000]
73     354.16       A1      2.56249    site1 [0.90249, 0.59751, 0.50000]       site1 [0.09751, 0.40249, 0.50000]
74     354.24       A1      2.56249    site1 [0.09751, 0.40249, 0.50000]       site1 [0.90249, 0.59751, 0.50000]
75     355.76       A2      2.80311    site1 [0.90249, 0.59751, 0.50000]       site1 [0.84751, 0.34751, 0.33333]
76     372.4        A2      2.80311    site1 [0.84751, 0.34751, 0.33333]       site1 [0.09751, 0.40249, 0.50000]
77     372.63       A2      2.80311    site1 [0.09751, 0.40249, 0.50000]       site1 [0.84751, 0.34751, 0.33333]
78     377.34       A2      2.80311    site1 [0.84751, 0.34751, 0.33333]       site1 [0.90249, 0.59751, 0.16667]
79     377.95       A2      2.80311    site1 [0.90249, 0.59751, 0.16667]       site1 [0.84751, 0.84751, 0.00000]
80     380.76       A1      2.56249    site1 [0.84751, 0.84751, 0.00000]       site1 [0.65249, 0.65249, 0.00000]
81     389.88       A2      2.80311    site1 [0.65249, 0.65249, 0.00000]       site1 [0.90249, 0.59751, 0.83333]
82     392.46       A2      2.80311    site1 [0.90249, 0.59751, 0.83333]       site1 [0.65249, 0.65249, 0.00000]
83     392.69       A1      2.56249    site1 [0.65249, 0.65249, 0.00000]       site1 [0.84751, 0.84751, 0.00000]
84     393.15       A1      2.56249    site1 [0.84751, 0.84751, 0.00000]       site1 [0.65249, 0.65249, 0.00000]
85     393.91       A1      2.56249    site1 [0.65249, 0.65249, 0.00000]       site1 [0.84751, 0.84751, 0.00000]
86     396.95       A1      2.56249    site1 [0.84751, 0.84751, 0.00000]       site1 [0.65249, 0.65249, 0.00000]
87     398.16       A1      2.56249    site1 [0.65249, 0.65249, 0.00000]       site1 [0.84751, 0.84751, 0.00000]
88     400.6        A1      2.56249    site1 [0.84751, 0.84751, 0.00000]       site1 [0.65249, 0.65249, 0.00000]
89     401.28       A1      2.56249    site1 [0.65249, 0.65249, 0.00000]       site1 [0.84751, 0.84751, 0.00000]
90     401.43       A1      2.56249    site1 [0.84751, 0.84751, 0.00000]       site1 [0.65249, 0.65249, 0.00000]
91     402.19       A1      2.56249    site1 [0.65249, 0.65249, 0.00000]       site1 [0.84751, 0.84751, 0.00000]
92     405.61       A1      2.56249    site1 [0.84751, 0.84751, 0.00000]       site1 [0.65249, 0.65249, 0.00000]
93     406.37       A1      2.56249    site1 [0.65249, 0.65249, 0.00000]       site1 [0.84751, 0.84751, 0.00000]
94     408.65       A1      2.56249    site1 [0.84751, 0.84751, 0.00000]       site1 [0.65249, 0.65249, 0.00000]
95     409.18       A1      2.56249    site1 [0.65249, 0.65249, 0.00000]       site1 [0.84751, 0.84751, 0.00000]
96     410.63       A1      2.56249    site1 [0.84751, 0.84751, 0.00000]       site1 [0.65249, 0.65249, 0.00000]
97     412.91       A1      2.56249    site1 [0.65249, 0.65249, 0.00000]       site1 [0.84751, 0.84751, 0.00000]
98     417.09       A1      2.56249    site1 [0.84751, 0.84751, 0.00000]       site1 [0.65249, 0.65249, 0.00000]
99     419.29       A1      2.56249    site1 [0.65249, 0.65249, 0.00000]       site1 [0.84751, 0.84751, 0.00000]
100    419.29       A2      2.80311    site1 [0.84751, 0.84751, 0.00000]       site1 [0.90249, 0.09751, 0.83333]
101    425.68       A1      2.56249    site1 [0.90249, 0.09751, 0.83333]       site1 [0.09751, 0.90249, 0.83333]
102    431.91       A2      2.80311    site1 [0.09751, 0.90249, 0.83333]       site1 [0.34751, 0.84751, 0.66667]
103    461.62       A3      2.96709    site1 [0.34751, 0.84751, 0.66667]       site1 [0.34751, 0.84751, 0.33333]
104    485.11       A2      2.80311    site1 [0.34751, 0.84751, 0.33333]       site1 [0.09751, 0.90249, 0.16667]
105    505.32       A2      2.80311    site1 [0.09751, 0.90249, 0.16667]       site1 [0.15249, 0.65249, 0.33333]
106    554.5        A2      2.80311    site1 [0.15249, 0.65249, 0.33333]       site1 [0.40249, 0.59751, 0.16667]
107    571.9        A2      2.80311    site1 [0.40249, 0.59751, 0.16667]       site1 [0.15249, 0.65249, 0.00000]
====================================================================================================================
```

---
### Visualizing Vacancy Hopping Trajectories

`VacHopPy` provides three powerful methods for visualizing the vacancy hopping trajectories, each offering a unique perspective on the diffusion process.

* **`plot_vacancy_trajectory()`**

    This method generates a static **interactive 3D plot** of the complete vacancy paths.

* **`animate_occupation()`**

    This method creates a **GIF animation** showing the time evolution of **site occupations**. It visualizes how individual atoms move between lattice sites and, as a consequence, how vacancies propagate through the structure. 

* **`animate_vacancy_trajectory()`**

    This method generates an **interactive HTML-based video** of the vacancy trajectories.

In [31]:
calc.plot_vacancy_trajectory(vacancy_indices=[0, 1], unwrap=True)

### Animation : Site Occupation

In [37]:
calc.animate_occupation(step_init=5516, step_final=5518, dpi=100, fps=5, update_alpha=0)

Make Animation:   0%|                              | 0/2 [00:00<?, ?it/s]


Merging 2 snapshots into a GIF...
Successfully created 'occupation_video.gif'.

Execution Time: 2.579 seconds
Peak RAM Usage: 0.021 GB


In [12]:
for k, v in calc.transient_vacancy.items():
    if len(v) > 0:
        print(f"Step {k} ({k * calc.t_interval} ps) :  Site Index = {v}")
        print(f"  - Fractoinal Coord = {calc.lattice_sites[v][0]}")
        print(f"  - Cartesian Coord  = {calc.lattice_sites_cart[v][0]}")

Step 5516 (419.216 ps) :  Site Index = [20]
  - Fractoinal Coord = [0.9024922  0.0975078  0.83333333]
  - Cartesian Coord  = [8.38535446 0.90597733 7.41771258]


#### Interactive Animation

In [16]:
calc.animate_vacancy_trajectory(vacancy_indices=[0, 1], step_init=5500, step_final=5525, unwrap=False)

Make Animation: 100%|##############################| 26/26 [00:00<00:00, 77.40it/s]



'trajectory_video.html' created.

Execution Time: 0.626 seconds
Peak RAM Usage: 0.035 GB
